In [ ]:
%run "catalog_common.py"
ID_header('Open-FF:  CASNumber and IngredientName pairs', incl_links=True)
set_page_param()

Starting with the Open-FF version 10, we have used a curation method to better clean the FracFocus data set.  In this method, all unique combinations of CASNumber and IngredientName pairs are evaluated manually to determine the best chemical label to assign to records.  While FracFoucs records about 1,300 unique chemical materials, because of the numerous ways companies record these chemicals, this list of CASNumber/IngredientName pairs is over 25,000 entries long.

This is essentially a tranlation table.  The input is the CASNumber and IngredientName and the output is bgCAS, our best guess for the proper identity of the chemical in the record.  

See the bottom of this page for notes about issues with these pairs.

In [ ]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt

In [ ]:
#import core.Analysis_set as ana_set
#import core.Construct_set as const_set
import matplotlib.pyplot as plt
%matplotlib inline

df_cas = ana_set.Catalog_set(repo = repo_name, outdir='../common/').get_set(verbose=False)

In [ ]:
gb = df_cas.groupby(['CASNumber','IngredientName'],as_index=True)['UploadKey'].count().reset_index()
gb = gb.rename({'UploadKey':'record_count'},axis=1)

In [ ]:
casing = grd.get_curated_df(repo_name,'casing_curated.csv')
CAScurated = grd.get_curated_df(repo_name,'CAS_curated.csv').rename({'comment':'CAS_comment'},axis=1)
CAScurated.CAS_comment.fillna(' ',inplace=True)
casing = pd.merge(casing,CAScurated[['CASNumber','CAS_comment']],on='CASNumber',how='left')
#casing.columns

In [ ]:
gb1 = df_cas.groupby(['CASNumber','IngredientName'],as_index=True)['bgIngredientName'].first().reset_index()

casing = pd.merge(casing,gb,on=['CASNumber','IngredientName'],how='left')
casing = pd.merge(casing,gb1,on=['CASNumber','IngredientName'],how='left')
casing['<CAS'] = '<h1>||</h1>'
casing['<Ing'] = '<h1>||</h1>'
casing.CASNumber = '<b>'+casing.CASNumber+'</b>'
casing.IngredientName = '<b>'+casing.IngredientName+'</b>'
casing['curCAS'] = casing.curatedCAS+'<br>'+casing.categoryCAS
casing['curING'] = casing.prospect_CAS_fromIng+'<br>'+casing.syn_code
casing['curfinal'] = '<h3>'+casing.bgCAS+'</h3><br>'+casing.bgSource
#casing.comment.fillna('',inplace=True)

|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
| | **Curation of CASNumber**|
|*raw CASNumber*| is the text in the CASNumber field of the original FracFocus data set, as found.|
|*CAS comment:*| any comments made the the Open-FF person evaluating the raw CASNumber.|  
|*curated CASNumber*| is the CAS number after curation; below is the category of action taken.|
| | **Curation of IngredientName**|
|*IngredientName*| is the raw text in the IngredientName field of the original FracFocus data set, as found.|
|*implied CAS from IngName (code)*| is the implied CAS number from the IngredientName as given; below is the curation code|
| | **Curation of the CASNumber/IngredientName Pair**|
|*final bgCAS and source*| shows the 'best guess' CAS Number when considering the curated version of CASNumber and IngredientName. Below shows which of the pair were used for this conclusion.|
|*record_count*| is the number of times this CASNumber/IngredientName pair occurs in the original FracFocus data set.|


In [ ]:
casing = casing[casing.record_count.notna()][['CASNumber','CAS_comment','curCAS','<CAS',
                                             'IngredientName','curING','<Ing',
                                             'curfinal','record_count']]
casing = casing.rename({'curCAS':'curated CASNumber','CASNumber':'raw CASNumber',
                 'curING':'implied CAS from IngName (code)','curfinal':'final bgCAS and source',
                 'bgCAS':'output: bgCAS','record_count':'record count'},axis=1)
iShow(casing.reset_index(drop=True),maxBytes=0)
# iShow(casing,maxBytes=0)


## Problems with the original FracFocus entries
If FracFocus had consistent quality-control, this translation table would probably not be needed.  Nevertheless, here are some of the problems we must address:

### - many names for a single chemical
This is not necessarily a 'problem.'  Most chemicals have more than one name associated with them.  These 'synonyms', (for example, 'baking soda,' 'bicarbonate of soda,' and 'sodium bicarbonate') all refer to the same substance.  In FracFocus, alternative and colloquial names and creative spellings are present.  

### - CASNumber and IngredientName are inconsistent
A particularly striking feature of the primary FracFocus data is the frequent mismatch between the CASNumber and the ingredient name given in a record. The FracFocus website addresses this issue by noting that there are often multiple names for any given chemical and it therefore advises using the CASNumber as the ultimate reference for the record's chemical identity.  Unfortunately, the problem is larger than just synonyms for a given chemical;  the two sources of identity are often completely incompatible.  For example, for the given CASNumber of water (7732-18-5), we find these IngredientName values:  "polyethylene glycol," "polycarboxylate, sodium salt," and "sodium chlorate" and others. Still, some form of the correct name "water" is the majority value for 7732-18-5 records, but to filter out all inconsistent records would be a serious limitation.  So, we follow the advice of FracFocus: **use the CASNumber as the primary reference for chemical identity**.  (Below, we describe some unique situations when we ignore this advice.)

if the CASNumber is ambiguous or worse, it may be tempting to use just the IngredientName to make the call on the record's identity.  However, because of the unreliability of IngredientName throughout much of the data set, we avoid this practice, and give the bgCAS a non-authoritative value, essentially removing it from analysis data sets.  The exception to this is when there is enough information in **other** fields to make a reasonable call. 

### - typos in CASNumber
Some original CASNumbers are clearly mistakenly entered. For example, a record with an IngredientName of 'water' and a CASNumber of '7**3**32-18-5' clearly is supposed to be '7**7**32-18-5'.  These errors are common and are corrected manually in this table.  

### - multiple CAS numbers in a single record
Sometimes operators enter more that one CASNumber in a single record.  This is a misuse of the disclosure because quantity cannot be properly represented. These records are removed from analysis data sets.

### - Use of deprecated CASNumbers
Some records report a CASNumber that has been registered but is no longer used by the Chemical Abstract Service. When possible, we translate the deprecated number to its correct version.

### - Use of a wrong CASNumber for a material
This issue occurs when a mistake in a CASNumber results in a different but 'valid' CAS number.  These mistakes can be quite hard to find because we first have to suspect that the reported chemical is not what was intended.  This can also occur when the CASNumber used is very different than the proper one, but still valid.  Many of these are correctable through curation.

A comical example: Forge Energy, LLC, used the CAS code 50-28-2 for their silica records on 4 separate disclosures in 2013.  That's the CASNumber of **estradiol**, the primary female hormone!  They reported about 1,000,000 pounds in each disclosure.  As of August 2021, those records were not corrected (API Numbers: 42-003-45517, 42-003-45157, 42-003-44956, and 42-003-45201).

### - Proprietary claims
Built into FracFocus is the ability to claim a chemical as confidential and therefore hide its identity.  

## Categorizing the curation action
These codes are used to indicate what sort of curation action was taken on each record:
- **perfect match**: For the majority of reported chemicals, the CASNumber is a perfect match with an authoritative CAS registry number as referenced on SciFinder. In these cases, bgCAS is the same as CASNumber.
- **cleaned**: For CASNumbers that are not a perfect match, Open-FF tries to coerce the values into an authoritatinve number. This is done by removing extraneous characters, removing leading zeros, etc. For example, the string '(7732-18-5#)' is changed to '7732-18-5'.  Note that some characters are not printable in the table below, but still need to be removed so matches are consistent. 
- **corrected**
- **proprietary**
- **ambiguous**
- **multiple chem**
- **CAS in IngredientName**
- **deprecated but valid**
- **Non-CAS**: Includes records where CASNumber is 'missing.'